In [2]:
import warnings
from importlib import reload

with warnings.catch_warnings():
    ## ignore future warnings from too-advanced numpy version
    warnings.simplefilter("ignore")    
    from keras.models import Sequential
    from keras.layers import Dense, Activation
import pandas as pd, numpy as np

from sklearn import preprocessing

In [42]:
import energy
from energy import MissingStrategy
_ = reload(energy)

In [8]:
MAX_ROWS =8*1000*1000
try:
    combined_df = energy.Data.Merged(MAX_ROWS).data
except FileNotFoundError:
    weather = energy.Data.Weather(MAX_ROWS)
    meter = energy.Data.Meter(MAX_ROWS)
    buildings = energy.Data.Buildings()
    combined_df = energy.merge_data(
        weather_df=weather.train, 
        meter_df=meter.train,
        buildings_df=buildings.data)

In [6]:
## combined_df.to_csv(energy.Data.Merged.train_file,index=False)

In [9]:
combined_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [ ]:
weather_cols = [
    'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 
    'sea_level_pressure', 'wind_direction', 'wind_speed']
X_df = combined_df[[
    'square_feet', 'meter', 'year_built', 'primary_use'] +
    weather_cols]

le = preprocessing.LabelEncoder()
le.fit(list(X_df.loc[:, 'primary_use']))
X_df.loc[:, 'primary_use'] = le.transform(list(X_df['primary_use']))

In [11]:
X = X_df.values
y = combined_df[['meter_reading']].values
assert not np.isnan(y).any()

In [12]:
missing_strategy = energy.MissingStrategy.USE_COLUMN_MEAN
X, y = energy.handle_missing_predictor_values(X, y, missing_strategy)
assert not np.isnan(X).any()

In [13]:
X_scaled = preprocessing.scale(X)

In [27]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=X.shape[1]))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

In [43]:
rmsle = energy.define_rmsle(X_df.shape[0])
model.compile(optimizer='rmsprop',
              loss='mse', metrics=['mse', rmsle])

In [ ]:
model.fit(X_scaled, y, epochs=10, batch_size=10000)

Epoch 1/10
8000000/8000000 [==============================] - 10s 1us/step - loss: 42002068514.4200 - mse: 42002079744.0000 - rmsle: 0.1589
Epoch 2/10
8000000/8000000 [==============================] - 10s 1us/step - loss: 41989273606.5200 - mse: 41989259264.0000 - rmsle: 0.1593
Epoch 3/10
8000000/8000000 [==============================] - 10s 1us/step - loss: 41975853777.8000 - mse: 41975844864.0000 - rmsle: 0.1596
Epoch 4/10
8000000/8000000 [==============================] - 10s 1us/step - loss: 41962462621.7800 - mse: 41962487808.0000 - rmsle: 0.1597
Epoch 5/10
8000000/8000000 [==============================] - 9s 1us/step - loss: 41946741775.6000 - mse: 41946767360.0000 - rmsle: 0.1597
Epoch 6/10
8000000/8000000 [==============================] - 9s 1us/step - loss: 41929068996.6000 - mse: 41929076736.0000 - rmsle: 0.1597
Epoch 7/10
8000000/8000000 [==============================] - 10s 1us/step - loss: 41906991904.0600 - mse: 41906999296.0000 - rmsle: 0.1596
Epoch 8/10
7500000/800